# xbatcher / pytorch dataloader turtorial

If you have not installed xbatcher, can be done via pip or conda. The docs for xbatcher are here:

https://xbatcher.readthedocs.io/en/latest/

In [1]:
# !pip install xbatcher

# or

# conda install -c conda-forge xbatcher

In [2]:
import numpy as np

import xarray as xr
import xbatcher

In [3]:
# pytorch specfic loaders
import torch

from torch.utils.data import TensorDataset, DataLoader

In [4]:
# nice way to show imported packages
from watermark import watermark
%load_ext watermark

In [5]:
%watermark --iversions

xarray  : 2022.11.0
torch   : 1.13.0
xbatcher: 0.3.0
numpy   : 1.23.5



## xarray:

This is the standard toy dataset for xarray testing

In [6]:
ds = xr.tutorial.load_dataset("air_temperature")
print('number of timesteps:', len(ds.time))

# adding a second variable to be somewhat more realistic than a 1 variable dataset
ds['air2'] = ds.air+42.42 #nonsense numbers

ds

number of timesteps: 2920


<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 283.6 284.9 285.9 ... 338.9 338.6 338.1
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## xbatcher intro:

Note: the xbatcher turtorial uses data arrays, this is using the dataset

In [7]:
# number of batches, adjust this!
number_of_batches = 13

div = np.round((len(ds.time))/number_of_batches, 3)
div

224.615

In [8]:
bgen = xbatcher.BatchGenerator(ds, {'time': number_of_batches})

for batch in bgen:
    pass
batch

<xarray.Dataset>
Dimensions:  (time: 13, sample: 1325)
Coordinates:
  * time     (time) datetime64[ns] 2014-12-26T18:00:00 ... 2014-12-29T18:00:00
  * sample   (sample) object MultiIndex
  * lat      (sample) float32 75.0 75.0 75.0 75.0 75.0 ... 15.0 15.0 15.0 15.0
  * lon      (sample) float32 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Data variables:
    air      (sample, time) float32 242.9 242.1 242.8 ... 297.7 296.3 296.3
    air2     (sample, time) float32 285.3 284.5 285.2 ... 340.1 338.7 338.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [9]:
print('Is the first batch and the last batch the same size, even if not divided cleanly')
len(batch.isel(time=0).sample) == len(batch.isel(time=number_of_batches-1).sample)

Is the first batch and the last batch the same size, even if not divided cleanly


True

This behavior is explained in the xbatcher docs [here](https://xbatcher.readthedocs.io/en/latest/demo.html#Controlling-the-size/shape-of-batches) .

### pytorch stuff

PyTorch data loader info: https://pytorch.org/docs/stable/data.html

you can also define batch sizes in pytorch, but will have less control/might not scale as well. We will not be using this feature.

Worth poking around with to_array() to understand how it's dumping out the np arrays

In [10]:
np.shape(batch.to_array().values)

(2, 1325, 13)

In [11]:
a_tensor = torch.from_numpy(batch.to_array().values)

In [12]:
data_loader = DataLoader(TensorDataset(a_tensor), 
                         batch_size=1, 
                         shuffle=False)

In [13]:
data_loader

This dataset should probably be processed, normalized, etc., and maybe a differnt shape would be better for your use case. But this should serve as an example for a relatively simple case to go from a multi-variable xarray dataset to a pytorch data loader. 